<a href="https://colab.research.google.com/github/NMashalov/FederationLearning/blob/master/Fed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code was highly inspired by implementation in paper's repository:
https://github.com/hongliny/FedAc-NeurIPS20/tree/master


a9a was presented in article
https://arxiv.org/abs/2006.08950

In [1]:
!wget https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a9a

--2023-11-15 21:45:39--  https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a9a
Resolving www.csie.ntu.edu.tw (www.csie.ntu.edu.tw)... 140.112.30.26
Connecting to www.csie.ntu.edu.tw (www.csie.ntu.edu.tw)|140.112.30.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2329875 (2.2M)
Saving to: ‘a9a’

a9a                 100%[===================>]   2.22M  1.56MB/s    in 1.4s    

2023-11-15 21:45:41 (1.56 MB/s) - ‘a9a’ saved [2329875/2329875]



Dataset loading

In [7]:
from sklearn.datasets import load_svmlight_file

X_train, y_train = load_svmlight_file('a9a')

X_train = X_train.toarray()

In [ ]:
def broadcast_avg(self, pool):
        """
        Helper functions for FedAc and FedAvg, average and broadcast the weights.
        """
        avg = pool.mean(axis=0)
        pool = np.repeat(avg[np.newaxis, :], pool.shape[0], axis=0)
        return pool

In [ ]:
import numpy as np

def sample_grad(self, weight, batch_size):
        """
        Sample minibatch gradients with size batch_size (For MB-SGD and MB-AC-SGD)

        Arguments:
            weight: w
            batch_size: number of samples queried to compute the sample gradient.

        Return:
            Averaged gradients
        """
        samples_idx = np.random.choice(self.n_samples, batch_size, replace=True)
        X_sampled = self.X_train[samples_idx, :]
        y_sampled = self.y_train[samples_idx]

        grad = -((y_sampled * (1 - sigm((X_sampled @ weight) * y_sampled))).T @ X_sampled) / \
            len(y_sampled) + self.lambd * weight

        return grad

In [ ]:
def fedavg(self, eta, M, K, T, local_batch, record_intvl=512, print_intvl=8192, SEED=0):
        """
        Simulate Federated Averaging (FedAvg, a.k.a. Local-SGD, or Parallel SGD, etc.)

        Arguments:
            eta:    learning rate
            M:      number of workers
            K:      synchronization interval, (i.e., local steps)
            T:      total parallel runtime
            record_intvl:   compute the population loss every record_intvl steps.

        Return:
            A pandas.Series object of population loss evaluated.
        """
        np.random.seed(SEED)
        common_init_w = np.random.randn(*self.weight_shape)
        w_pool = np.repeat(common_init_w[np.newaxis, :], M, axis=0)

        seq = pd.Series(name='loss')
        for iter_cnt in range(T+1):
            if iter_cnt % K == 0:
                w_pool = self.broadcast_avg(w_pool)

                if iter_cnt % record_intvl == 0:
                    seq.at[iter_cnt] = self.population_loss(w_pool[0, :])

            w_pool -= eta * self.sample_grad_pool(w_pool, M, local_batch)
        return seq

In [ ]:
 def fedac(self, eta, gamma, alpha, beta, M, K, T, local_batch, record_intvl=512, print_intvl=8192, SEED=0):
        """
        Simulate FedAc

        Arguments:
            eta:    learning rate
            gamma, alpha, beta:  hyperparameters
            M:      number of workers
            K:      synchronization interval
            T:      total parallel runtime
            record_intvl:   compute the population loss every record_intvl steps.

        Return:
            A pandas.Series object of population loss evaluated.
        """
        np.random.seed(SEED)
        common_init_w = np.random.randn(*self.weight_shape)
        w_pool = np.repeat(common_init_w[np.newaxis, :], M, axis=0)
        w_ag_pool = np.copy(w_pool)

        seq = pd.Series(name='loss')

        for iter_cnt in range(T+1):
            if iter_cnt % K == 0:
                w_pool = self.broadcast_avg(w_pool)
                w_ag_pool = self.broadcast_avg(w_ag_pool)

                if iter_cnt % record_intvl == 0:
                    seq.at[iter_cnt] = self.population_loss(w_ag_pool[0, :])

            w_md_pool = (1/beta) * w_pool + (1-(1/beta))*w_ag_pool
            grad_md_pool = self.sample_grad_pool(w_md_pool, M, local_batch)
            w_ag_pool = w_md_pool - eta * grad_md_pool
            w_pool = (1 - (1/alpha)) * w_pool + (1/alpha) * \
                w_md_pool - gamma * grad_md_pool


In [ ]:
def loss(self, X, Y, weight):
        """
        Compute the loss.
        """
        return -np.mean(np.log(sigm((X @ weight) * Y))) + 0.5 * self.lambd * np.linalg.norm(weight) ** 2

In [ ]:
seq = pd.Series(name='loss')
    for iter_cnt in range(0, T+1, K):
        if iter_cnt % record_intvl == 0:
            seq.at[iter_cnt] = self.population_loss(w)
        w -= eta * self.sample_grad(w, M*K*local_batch)
    return seq